# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra
using ASEconvert

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al; psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]
    unit_cell = periodic_system(lattice, atoms, positions)

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE, make the supercell
    # and then convert back ...
    supercell_ase = convert_ase(unit_cell) * pytuple((repeat, 1, 1))
    supercell     = pyconvert(AbstractSystem, supercell_ase)

    # Unfortunately right now the conversion to ASE drops the pseudopotential information,
    # so we need to reattach it:
    supercell = attach_psp(supercell; Al="hgh/lda/al-q3")

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(supercell; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

    CondaPkg Found dependencies: /home/runner/.julia/packages/ASEconvert/CNQ1A/CondaPkg.toml
    CondaPkg Found dependencies: /home/runner/.julia/packages/PythonCall/qTEA1/CondaPkg.toml
    CondaPkg Dependencies already up to date


As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
convert_ase(periodic_system(setup.model)).write("al_supercell.png")

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123


Python: None

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298589646677                   -0.85    5.1         
  2   -8.300211843620       -2.79       -1.25    1.2    119ms
  3   -8.300434922444       -3.65       -1.88    2.0    180ms
  4   -8.300462002070       -4.57       -2.77    2.9    149ms
  5   -8.300464389784       -5.62       -3.12    4.2    176ms
  6   -8.300464520164       -6.88       -3.27    7.9    263ms
  7   -8.300464582278       -7.21       -3.41    1.2    137ms
  8   -8.300464615248       -7.48       -3.56    2.1    158ms
  9   -8.300464635691       -7.69       -3.75    1.5    186ms
 10   -8.300464639229       -8.45       -3.85    1.0    119ms
 11   -8.300464643290       -8.39       -4.12    2.9    173ms


In [4]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -16.67447173309                   -0.70    5.9         
  2   -16.67863098196       -2.38       -1.14    1.4    290ms
  3   -16.67921185055       -3.24       -1.87    4.1    393ms
  4   -16.67927419957       -4.21       -2.76    3.2    366ms
  5   -16.67928588329       -4.93       -3.17    3.9    409ms
  6   -16.67928619621       -6.50       -3.48    2.4    317ms
  7   -16.67928621648       -7.69       -3.94    1.8    307ms
  8   -16.67928622140       -8.31       -4.57    3.0    340ms


In [5]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32378529847                   -0.56    7.0         
  2   -33.33251978126       -2.06       -1.00    1.2    1.09s
  3   -33.33405709678       -2.81       -1.74    3.4    1.34s
  4   -33.33427501202       -3.66       -2.65    4.1    1.32s
  5   -33.33577476602       -2.82       -2.33    5.4    1.65s
  6   -33.33694199429       -2.93       -2.52    8.2    1.56s
  7   -33.33694207214       -7.11       -2.52    1.0    1.11s
  8   -33.33692404510   +   -4.74       -2.57    2.4    1.21s
  9   -33.33684395693   +   -4.10       -2.32    1.2    1.11s
 10   -33.33684570652       -5.76       -2.32    1.0    1.10s
 11   -33.33684672210       -5.99       -2.33    1.0    1.07s
 12   -33.33685262982       -5.23      

When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298349096016                   -0.85    5.4         
  2   -8.300257696027       -2.72       -1.59    1.0    108ms
  3   -8.300424083446       -3.78       -2.55    3.1    123ms
  4   -8.300380224155   +   -4.36       -2.29    8.5    255ms
  5   -8.300464252206       -4.08       -3.38    1.1   97.7ms
  6   -8.300464577884       -6.49       -3.81    3.2    159ms
  7   -8.300464640327       -7.20       -4.24    1.2    116ms


In [7]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32445965963                   -0.56    7.2         
  2   -33.32731237320       -2.54       -1.27    1.0    874ms
  3   -24.56863119062   +    0.94       -0.60    5.6    1.67s
  4   -33.16944302672        0.93       -1.41    6.8    1.95s
  5   -33.23885122598       -1.16       -1.33    2.6    1.29s
  6   -32.12000719561   +    0.05       -1.02    4.2    1.40s
  7   -33.32692532366        0.08       -2.00    4.5    1.31s
  8   -33.33223715496       -2.27       -2.16    2.8    1.28s
  9   -33.33562515336       -2.47       -2.29    2.1    1.16s
 10   -33.33569498248       -4.16       -2.46    1.8    1.04s
 11   -33.33661523624       -3.04       -2.75    1.8    992ms
 12   -33.33689542220       -3.55      

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.